In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
import json
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Killybegs,IE,54.6333,-8.4500,62.49,91,100,7.72,overcast clouds
1,1,Busselton,AU,-33.6500,115.3333,51.26,84,0,7.67,clear sky
2,2,Geraldton,AU,-28.7667,114.6000,52.21,93,27,11.50,scattered clouds
3,3,Salalah,OM,17.0151,54.0924,77.09,94,100,4.61,overcast clouds
4,4,Butaritari,KI,3.0707,172.7902,82.36,75,16,19.77,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 70
What is your desired maximum temperature for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Salalah,OM,17.0151,54.0924,77.09,94,100,4.61,overcast clouds
4,4,Butaritari,KI,3.0707,172.7902,82.36,75,16,19.77,few clouds
5,5,Vaini,TO,-21.2000,-175.2000,78.96,83,40,12.66,scattered clouds
9,9,Lorengau,PG,-2.0226,147.2712,82.40,73,97,6.22,light rain
11,11,Ribeira Grande,PT,38.5167,-28.7000,72.64,91,16,3.60,few clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Vila Velha,BR,-20.3297,-40.2925,71.55,85,0,3.44,clear sky
692,692,Regeneracao,BR,-6.2378,-42.6883,78.73,32,32,8.48,scattered clouds
694,694,Tebingtinggi,ID,3.3285,99.1625,75.09,96,97,3.02,overcast clouds
696,696,Sant Feliu De Guixols,ES,41.7833,3.0333,80.13,63,0,3.51,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Salalah,OM,17.0151,54.0924,77.09,94,100,4.61,overcast clouds
4,4,Butaritari,KI,3.0707,172.7902,82.36,75,16,19.77,few clouds
5,5,Vaini,TO,-21.2000,-175.2000,78.96,83,40,12.66,scattered clouds
9,9,Lorengau,PG,-2.0226,147.2712,82.40,73,97,6.22,light rain
11,11,Ribeira Grande,PT,38.5167,-28.7000,72.64,91,16,3.60,few clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Vila Velha,BR,-20.3297,-40.2925,71.55,85,0,3.44,clear sky
692,692,Regeneracao,BR,-6.2378,-42.6883,78.73,32,32,8.48,scattered clouds
694,694,Tebingtinggi,ID,3.3285,99.1625,75.09,96,97,3.02,overcast clouds
696,696,Sant Feliu De Guixols,ES,41.7833,3.0333,80.13,63,0,3.51,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
4,Butaritari,KI,82.36,few clouds,3.0707,172.7902,
5,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
9,Lorengau,PG,82.40,light rain,-2.0226,147.2712,
11,Ribeira Grande,PT,72.64,few clouds,38.5167,-28.7000,
13,Hithadhoo,MV,81.84,overcast clouds,-0.6000,73.0833,
15,Oregon City,US,81.81,clear sky,45.3573,-122.6068,
18,Lago Da Pedra,BR,79.84,broken clouds,-4.3333,-45.1667,
20,Biak,ID,75.11,overcast clouds,-0.9131,122.8766,
22,Gat,IL,79.39,clear sky,31.6100,34.7642,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    #f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")   

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
4,Butaritari,KI,82.36,few clouds,3.0707,172.7902,Isles Sunset Lodge
5,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Lorengau,PG,82.40,light rain,-2.0226,147.2712,Lorengau Harbourside Hotel
11,Ribeira Grande,PT,72.64,few clouds,38.5167,-28.7000,Quinta da Meia Eira
...,...,...,...,...,...,...,...
690,Vila Velha,BR,71.55,clear sky,-20.3297,-40.2925,Hotel Vitória Palace
692,Regeneracao,BR,78.73,scattered clouds,-6.2378,-42.6883,Hotel KANAYRA
694,Tebingtinggi,ID,75.09,overcast clouds,3.3285,99.1625,Hotel Malibou tebing tinggi
696,Sant Feliu De Guixols,ES,80.13,clear sky,41.7833,3.0333,Hotel NM Suites


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [4]:

# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

NameError: name 'clean_hotel_df' is not defined

In [3]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

NameError: name 'locations' is not defined